In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as pltlines
from pymatreader import read_mat
import cv2
class DraggableRectangleCenter:
    lock = None  # only one can be animated at a time

    def __init__(self, rect,drag_orth,dist):
        self.rect = rect
        self.press = None
        self.background = None
        self.drag_orth = drag_orth
        self.circle = None
        self.line = None
        self.dist=dist
    def connect(self):
        """Connect to all the events we need."""
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        """Check whether mouse is over us; if so, store some data."""
        if (event.inaxes != self.rect.axes
                or DraggableRectangleCenter.lock is not None):
            return
        contains, attrd = self.rect.contains(event)
        if not contains:
            return
        print('event contains', self.rect.xy)
        self.press = self.rect.xy, (event.xdata, event.ydata)
        DraggableRectangleCenter.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        """Move the rectangle if the mouse is over us."""
        if (event.inaxes != self.rect.axes
                or DraggableRectangleCenter.lock is not self):
            return
        (x0, y0), (xpress, ypress) = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        """Clear button press information."""
        if DraggableRectangleCenter.lock is not self:
            return

        self.press = None
        DraggableRectangleCenter.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()
        x,y = self.rect.xy
        if self.circle is not None:
            self.circle.remove()
            self.rect.figure.gca().lines.pop(0)
        x1,y1 = self.drag_orth.rect.xy
        circle = plt.Circle((x,y),1000,alpha = 0.3)
        direction = np.array((x1-x,y1-y))
        direction = direction/np.linalg.norm(direction)
        pos_line = np.array((x,y))+dist*direction
        # circle2 = plt.Circle((pos_line[0],pos_line[1]),100,alpha = 0.3)
        orth_direct = np.array([direction[1],-direction[0]])
        extension = 1000
        deb_line = pos_line + extension*orth_direct
        end_line = pos_line - extension*orth_direct
        line = pltlines.Line2D((deb_line[0],end_line[0]),(deb_line[1],end_line[1]),color='red')
        self.line = line
        self.circle = circle
        ax = self.rect.figure.gca()
        ax.add_patch(circle)
        # ax.add_patch(circle2)
        ax.add_line(line)
    def disconnect(self):
        """Disconnect all callbacks."""
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

class DraggableRectangleOrth:
    lock = None  # only one can be animated at a time

    def __init__(self, rect):
        self.rect = rect
        self.press = None
        self.background = None
        self.line = None
    def connect(self):
        """Connect to all the events we need."""
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        """Check whether mouse is over us; if so, store some data."""
        if (event.inaxes != self.rect.axes
                or DraggableRectangleOrth.lock is not None):
            return
        contains, attrd = self.rect.contains(event)
        if not contains:
            return
        print('event contains', self.rect.xy)
        self.press = self.rect.xy, (event.xdata, event.ydata)
        DraggableRectangleOrth.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        """Move the rectangle if the mouse is over us."""
        if (event.inaxes != self.rect.axes
                or DraggableRectangleOrth.lock is not self):
            return
        (x0, y0), (xpress, ypress) = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        """Clear button press information."""
        if DraggableRectangleOrth.lock is not self:
            return

        self.press = None
        DraggableRectangleOrth.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()
    def disconnect(self):
        """Disconnect all callbacks."""
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)


list_40 = [102,94,80,22,26,59,69,76,80,94]
list_65 = [808,800]
list_70 = [88,785,792]
im = read_mat('raw_image.mat')['raw']
shape_compressed = im.shape[1]//5,im.shape[0]//5
im_comp = cv2.resize(im,shape_compressed)
fig, ax = plt.subplots()

ax.imshow(im_comp)
dist = 150
# circle = plt.Circle((2000,2000),100,alpha = 0.3)
# ax.add_patch(circle)
rect_center = ax.bar(500, 25,25)[0]
rect_orth = ax.bar(1000, 25,25,color='red')[0]

dr_orth = DraggableRectangleOrth(rect_orth)
dr_orth.connect()
dr_center = DraggableRectangleCenter(rect_center,dr_orth,dist)
dr_center.connect()
plt.show()

OSError: could not read bytes